# Exploration and Exploitation
This lesson introduces the problem of multi-arms bandits: in front of you there are K arms, each one of them giving you a different average rewards. The goal is to maximize your cumulated rewards over a given number of actions (we have a limited budget of actions). The problem is not an easy one to solve as it requires to balance Exploration (ie testing new arms) and Exploitation (ie pulling the arm that gave us the biggest amount of rewards)

In this case there are only action, not states:

The action value function is defined by:
$$
Q(a) = \mathbb E [r|a]
$$

There are different approaches:
* A purely random one that gives us a baseline
* An $\epsilon$-greedy one that work well in practice
* An Upper Confidence Bound approach that uses uncertainty over the action